In [2]:
import ru_core_news_lg
import spacy as sp
import nltk
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

from function import *

nlp = sp.load('ru_core_news_lg')

#### **Глава 8. Распознавание намерений**

Начнем с распознавания намерений пользователя с помощью вы- деления переходного глагола и прямого дополнения высказывания. Далее обсудим, как выяснить намерение пользователя по последовательности предложений, как распознать синонимы, выражающие различные возможные намерения, и определить намерение пользователя на основе семантического подобия.


#### **Распознавание намерений с помощью выделения переходного глагола и прямого дополнения**

Обычно распознавание намерения пользователя состоит из трех этапов: 

1) синтаксического разбора предложения на токены
2) соединения токенов маркированными дугами, отражающими синтаксические отношения
3) проход по этим дугам для выделения соответствующих токенов.

Но во многих случаях для распознавания намерений пользователя **достаточно** выделить переходный глагол и прямое дополнение, как видно из разбора синтаксических зависимостей на картинке

In [42]:
Image(url="images/picture_12.png", width=500, height=250)

Дуга, соединяющая переходный глагол с прямым дополнением, указывает, что пользователь намерен найти гостиницу (если объединить переходный глагол и прямое дополнение в одно слово, получится findHotel). В последующем коде эту структуру можно использовать в качестве **идентификатора намерения** (intent identifier) - они будут  рассмотрены в Главе 11!

#### **Получение пары «переходный глагол/прямое дополнение»**

In [43]:
doc = nlp(u'найди мне лучший отель в берлине.')

displacy.render(doc, style='dep')

for token in doc:
    if token.dep_ == 'obj':
        print(token.head.text + token.text.capitalize())

найдиОтель


#### **Выделение множественных намерений с помощью token.conjuncts**

Иногда встречаются предложения, выражающие несколько намерений. Например, такое:

- Я хочу книгу и чай.

В большинстве случаев эти намерения можно считать частью одного составного намерения. 

In [44]:
doc = nlp(u'Я хочу книгу и чай.')
displacy.render(doc, style='dep')

На схеме есть две стрелки, указывающие на дуги прямого дополнения *книгу* и его конъюнкта *чай*. **Конъюнкт** (conjunct) существительного — это другое существительное, присоединенное к первому посредством союза, такого как «и», «или» и т. д

In [45]:
doc = nlp(u"Я хочу книгу и чай.")

for token in doc:
    if token.dep_ == 'obj':
        obj = [token.text]
        #  пара “переходный глагол/прямое дополнение”
        verb = [token.head.text]
        conj = [t.text for t in token.conjuncts]
        
# Составляем список выделенных элементов
obj_conj = verb + obj + conj 
print(obj_conj)

['хочу', 'книгу', 'чай']


In [46]:
for token in doc:
    if token.dep_ == 'obj':
        obj = [token.text]
        #  пара “переходный глагол/прямое дополнение”
        verb = [token.head.text]
        conj = [t.text for t in token.rights if t.dep_ == 'conj']
        
# Составляем список выделенных элементов
obj_conj = verb + obj + conj 
print(obj_conj)

['хочу', 'книгу', 'чай']


#### **Выделение намерения с помощью списков слов**
В некоторых случаях намерение пользователя лучше всего описывают не переходный глагол и прямое дополнение, а другие токены. Обычно они связаны отношением с переходным глаголом или прямым дополнением: просто нужно будет сделать еще один шаг и найти слова, лучше всего описывающие намерение, путем исследования синтаксических отношений переходного глагола и прямого дополнения

In [47]:
doc = nlp(u"Я хочу сделать заказ книг, чай и кофе.")

displacy.render(doc, style='dep')

Для выделения слов **хочу** и **книг** из высказывания воспользуемся списком заранее определенных слов и найдем их в высказываний.

Может показаться, что это будет не достаточно эффективным, однако, в случае "заказов" такой метод достаточно хорош.

In [48]:
# Выделяем прямое дополнение и его переходный глагол
obj = ''
tverb = ''
for token in doc:
    if token.dep_ == 'obj':
        obj = token
        tverb = token.head

print("Прямое дополнение: %s\nПереходный глагол: %s " % (obj, tverb))

# Выделяем глагол для описания намерения
intentVerb = ''
verbList = ['хочу', 'нравится', 'нужно', 'заказ']
if tverb.text in verbList:
    intentVerb = tverb
else:
    if tverb.head.dep_ == 'ROOT':
        intentVerb = tverb.head

print("Глагол для описания намерения: ", intentVerb)

# Выделяем дополнение для описания намерения
intentObj = []
objList = ['книг', 'чай', 'кофе']
if obj.text in objList: 
    intentObj = obj
else:
    for child in obj.children:
        if child.dep_ == 'nmod':
            intentObj.append(child)
            intentObj += list(child.children)[::]
            break
        elif child.dep_ == 'compound':
            intentObj = child
            break
all = ''
for t in intentObj:
    all+= t.text.capitalize() 

# Выводим в консоль высказанное в образце предложения намерение
print(intentVerb.text + all)

Прямое дополнение: заказ
Переходный глагол: сделать 
Глагол для описания намерения:  хочу
хочуКнигЧайКофе


#### **Поиск значений слов с помощью синонимов и семантического подобия**
Существует несколько подходов к распознаванию синонимов.

Один из них — использовать заранее заданные списки синонимов, искать в них нужные слова и определять значение слов в соответствии с тем, в каком списке они оказались.

Еще один способ — распознавать синонимы по степени семантического подобия.

#### **Распознавание синонимовс помощью заранее заданных списков**
Простейший способ выяснить, сколько намерений выражают две фразы — одно или два, — проверить, **синонимичны** ли в них переходные глаголы и прямые дополнения.

Например, эти три фразы выражают одно намерение - заказПиццы:
- Я хочу блюдо. Я бы хотел заказать пиццу. Дай мне пирог.

Обработка этих высказываний состоит из следующих шагов.
1. Разбор зависимостей для выделения из предложения переходного глагола и его прямого дополнения.
2. Поиск переходного глагола и его прямого дополнения в заранее заданных списках синонимов и замена (в случае успеха) их словами, понятными приложению.
3. Составление строки, идентифицирующей намерение.


*Распознавание намерения с помощью списков синонимов*:

In [49]:
Image(url="images/picture_13.png", width=500, height=450)

In [50]:
# Применяем конвейер к примеру предложения
doc = nlp(u'Я хочу блюдо.')

displacy.render(doc, style='dep')

In [51]:
# Выделяем из дерева зависимостей переходный глагол и его прямое дополнение
for token in doc:
    if token.dep_ == 'obj':
        verb = token.head.text
        obj = token.text
# Создаем список кортежей возможных синонимов глагола
verbList = [('заказ','хочу','дай','сделать'), ('покажи','найди')]

# Находим кортеж с выделенным из примера текста переходным глаголом 
verbSyns = [item for item in verbList if verb in item]
print(verbSyns)

# Создаем список кортежей возможных синонимов прямого дополнения
dobjList = [('пицца','пирог','блюдо'), ('кола','газировка')]

# Находим кортеж с выделенным из примера текста прямым дополнением
dobjSyns = [item for item in dobjList if obj in item]
print(dobjSyns)

# Заменяем переходный глагол и прямое дополнение понятными приложению  синонимами и формируем строку-идентификатор намерения
intent = verbSyns[0][0] + dobjSyns[0][0].capitalize()

print(intent)

[('заказ', 'хочу', 'дай', 'сделать')]
[('пицца', 'пирог', 'блюдо')]
заказПицца


Cценарий

In [52]:
def sinonim(doc):
    for token in doc:
        if token.dep_ == 'obj':
            verb = token.head.text
            obj = token.lemma_
    print(verb, obj)
            
    verbList = [('заказ','хочу','дай','сделать', 'заказать'), ('покажи','найди')]
    verbSyns = [item for item in verbList if verb in item]
    print(verbSyns)

    dobjList = [('пицца','пирог','блюдо'), ('кола','газировка')]
    dobjSyns = [item for item in dobjList if obj in item]
    print(dobjSyns)

    if verbSyns == [] or dobjSyns == []:
        return "Намерение не распознано"
    else:
        return verbSyns[0][0] + dobjSyns[0][0].capitalize()
    

In [53]:
print(sinonim(nlp(u'Я хочу яблоко.')))

хочу яблоко
[('заказ', 'хочу', 'дай', 'сделать', 'заказать')]
[]
Намерение не распознано


In [54]:
print(sinonim(nlp(u'Я хочу заказать пиццу.')))

заказать пицца
[('заказ', 'хочу', 'дай', 'сделать', 'заказать')]
[('пицца', 'пирог', 'блюдо')]
заказПицца


#### **Распознавание неявных намерений с помощью семантического подобия**

Один из широко известных способов заставить пользователя выразить намерение яснее — задать уточняющий вопрос. Для выбора задаваемого вопроса необходимо вычислить семантическое подобие предыдущего высказывания пользователя.

**Первый шаг** — анализ дерева зависимостей входного высказывания и выделение прямого дополнения и его переходного глагола. Если прямое дополнение не удалось найти в заранее заданном списке синонимов, можно попытаться определить, насколько оно близко к включенным в этот список словам.

После этого на основе вычисления семантического подобия можно будет сгенерировать уточняющий вопрос пользователю.

In [55]:
Image(url="images/picture_14.png", width=550, height=450)

In [56]:
doc = nlp(u'Мне хочется съесть пирог.')

for token in doc:
    if token.dep_ == 'obj':
        obj = token

tokens = nlp(u'еда')
print(obj.similarity(tokens[0]))

if obj.similarity(tokens[0]) > 0.34:
    question = 'Хотите ознакомиться с нашим меню?'
    print(question)

0.36646491289138794
Хотите ознакомиться с нашим меню?


**Попробуйте сами**

Конечно, нам, как и нашему приложению, заранее не известно, какие фразы применит пользователь и насколько легко будет распознать его намерения. Именно поэтому на практике в приложениях сочетается несколько подходов для распознавания намерения. Для обработки более широкого спектра случаев попробуйте совместить подход на основе распознавания синонимов и подход, основанный на выявлении неявных намерений. Сначала ваш код должен попытаться определить намерение высказывания с помощью подхода, основанного на распознавании синонимов, а если не получилось — использовать и подход на основе семантического подобия. Если же результат все равно не достигнут, высказывание можно пометить как выражающее нераспознанное намерение.

In [57]:
def determinant_intentions(doc):
    tokens = nlp(u'еда')
    for token in doc:
        if token.dep_ == 'obj':
            verb = token.head.text
            obj = token
            
    verbList = [('заказ','хочу','дай','сделать', 'заказать'), ('покажи','найди')]
    verbSyns = [item for item in verbList if verb in item]
    print(verbSyns)

    dobjList = [('пицца','пирог','блюдо'), ('кола','газировка')]
    dobjSyns = [item for item in dobjList if obj.lemma_ in item]
    print(dobjSyns)

    if obj.similarity(tokens[0]) > 0.34:
        question = 'Хотите ознакомиться с нашим меню?'
        print("Quedtions:", question)
    elif verbSyns == [] or dobjSyns == []:
        print("Намерение не распознано")
    else:
        print(verbSyns[0][0] + dobjSyns[0][0].capitalize())

  
determinant_intentions(nlp(u'Мне хочется съесть пирог.'))

[]
[('пицца', 'пирог', 'блюдо')]
Quedtions: Хотите ознакомиться с нашим меню?


In [58]:
determinant_intentions(nlp(u'Я хочу яблоко.'))

[('заказ', 'хочу', 'дай', 'сделать', 'заказать')]
[]
Намерение не распознано


In [59]:
print(sinonim(nlp(u'Я хочу заказать пиццу.')))

заказать пицца
[('заказ', 'хочу', 'дай', 'сделать', 'заказать')]
[('пицца', 'пирог', 'блюдо')]
заказПицца


#### **Выделение намерения из последовательности предложений**


Слова, выражающие намерение пользователя, могут встречаться в различных предложениях связного текста. Например:

- Я доел свою пиццу. Я хочу еще одну.

В подобном случае чат-бот должен быть готов к выделению намерений пользователя из текста в целом. В этом разделе рассмотрим методики решения данной задачи.



#### **Обход структуры зависимостей связного текста**

Начнем с синтаксического разбора зависимостей текста с выявлением пар *«переходный глагол/прямое дополнение"* в каждом из предложений.

Серая стрелка на схеме соответствует зависимости, которая нас интересует. 

In [60]:
Image(url="images/picture_15.png", width=550, height=250)

#### **Замена местоименных элементов их антецедентами**

**Антецедент (antecedent)** — выражение (слово или простое предложение), обозначаемое местоименным элементом (pro-form), например местоимением или местоглаголием. 

При такой разновидности выделения намерений необходимо определить антецеденты и заменить ими соответствующие местоименные элементы. Для этого нужно проделать следующее:

1. Произвести синтаксический разбор зависимостей всего текста.
2. Разбить связный текст на предложения.
3. Найти антецедент (который далее укажем при формировании идентификатора намерения) для местоимения, служащего прямым дополнением переходного глагола.

In [61]:
Image(url="images/picture_16.png", width=550, height=500)

In [62]:
doc = nlp(u'Я доел свою пиццу. Я хочу еще одну.')

displacy.render(doc, style='dep')

verbList = [('заказ','хочу','дай','сделать', 'заказать', 'доел'), ('покажи','найди')]
dobjList = [('пицца','пирог','блюдо'), ('кола','газировка')]

#Список слов-заместителей
substitutes = ('одну','это','такую','еще')

# части описания намерений
intent = {'verb': '', 'obj': ''}

for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'obj':
            print(token)
            verbSyns = [item for item in verbList if token.head.text in item]
            print(verbSyns)

            print("token: ", token)
            objSyns = [item for item in dobjList if token.lemma_ in item] 
            print("objSyns: ", objSyns)

            substitute = [item for item in substitutes if token.text in item] 
            print("substitute: ", substitute)
    
            if (objSyns != [] or substitute != []) and verbSyns != []:
                intent['verb'] = verbSyns[0][0]
            if objSyns != []:
                intent['obj'] = objSyns[0][0]


пиццу
[('заказ', 'хочу', 'дай', 'сделать', 'заказать', 'доел')]
token:  пиццу
objSyns:  [('пицца', 'пирог', 'блюдо')]
substitute:  []


In [63]:
intentStr = intent['verb'] + intent['obj'].capitalize()

print(intentStr)

заказПицца
